In [1]:
import pandas as pd
import os,natsort,json

In [2]:
dir = "/mydata/rbachkaniwala3/code/rajveerb-ml-pipeline-benchmark/code/image_classification/analysis/hw_event_analysis/vtune_uarch_csvs"

In [3]:
# get all csv files in the directory
csv_files = [f for f in os.listdir(dir) if f.endswith('.csv')]
# natural sort the csv files
csv_files = natsort.natsorted(csv_files)

In [4]:
csv_files

['b128_gpu1.csv',
 'b128_gpu2.csv',
 'b128_gpu3.csv',
 'b128_gpu4.csv',
 'b256_gpu1.csv',
 'b256_gpu2.csv',
 'b256_gpu3.csv',
 'b256_gpu4.csv',
 'b512_gpu1.csv',
 'b512_gpu2.csv',
 'b512_gpu3.csv',
 'b512_gpu4.csv',
 'b1024_gpu2.csv',
 'b1024_gpu3.csv',
 'b1024_gpu4.csv']

In [5]:

dfs = {}
# read the csv file as a dataframe

for csv_file in csv_files:
    dfs[csv_file] = pd.read_csv(os.path.join(dir,csv_file),sep='\t')

In [6]:
dfs

{'b128_gpu1.csv':          Source Function / Function / Call Stack   CPU Time  \
 0                                     decode_mcu  2648.443s   
 1                                jpeg_idct_islow  2067.762s   
 2                 ImagingResampleHorizontal_8bpc   799.221s   
 3                                ycc_rgb_convert   732.101s   
 4                   ImagingResampleVertical_8bpc   565.856s   
 ..                                           ...        ...   
 320  at::TensorIteratorBase::coalesce_dimensions     1.210s   
 321                                   __tls_init     1.210s   
 322                                 do_task_stat     1.205s   
 323                            copy_page_to_iter     1.200s   
 324               c10::TensorImpl::requires_grad     1.195s   
 
             Clockticks Instructions Retired  CPI Rate Retiring  \
 0    6,335,216,000,000   10,622,896,000,000     0.596    40.7%   
 1    4,948,624,000,000   13,991,232,000,000     0.354    69.5%   
 2    2,172,

In [7]:
# need to clean up this block of code
csv_dir="/mydata/rbachkaniwala3/code/rajveerb-ml-pipeline-benchmark/code/image_classification/analysis/low_level_func/logs"
program_file_dir="/mydata/rbachkaniwala3/code/rajveerb-ml-pipeline-benchmark/code/image_classification/analysis/low_level_func"
# get all program files that end with .py
program_files = [f for f in os.listdir(program_file_dir) if f.endswith(".py")]
program = program_files[0].split(".")[0]
same_program_csv_dir=os.path.join(csv_dir,program)
# samples = total number of runs for each program
samples = len(os.listdir(same_program_csv_dir))
op_to_func = {}

# TODO need to change the intersection to count
# loop through csv files in csv_dir
for program_ in program_files:
    program = program_.split(".")[0]
    same_program_csv_dir=os.path.join(csv_dir,program)
    common_funcs_per_op = set()
    c_func_to_count = {}
    for csv_file in os.listdir(same_program_csv_dir):
        df = pd.read_csv(os.path.join(same_program_csv_dir,csv_file))
        # merge two columns with "|" as separator
        df["full_func"] = df['Function'].astype(str) + "|" + df['Module'].astype(str)
        # loop through each function in the column 'full_func'
        for func in df["full_func"].values:
            # if function is not in the dictionary, add it
            if func not in c_func_to_count:
                c_func_to_count[func] = 1
            # else increment the count
            else:
                c_func_to_count[func] += 1

    for func in c_func_to_count:
        # i.e. present in more than half of the samples
        # if c_func_to_count[func] > samples//2:
        common_funcs_per_op.add(func)        

    op_to_func[program] = list(common_funcs_per_op)
    print(f"Common functions for {program} are {common_funcs_per_op}")

Common functions for RandomHorizontalFlip are {'_PyEval_EvalFrameDefault|python3.10', 'c10::impl::OperatorEntry::lookup|libtorch_cpu.so', 'ImagingFlipLeftRight|_imaging.cpython-310-x86_64-linux-gnu.so', '__libc_calloc|libc.so.6', '__GI___pthread_mutex_lock|libpthread.so.0', '__GI___libc_malloc|libc.so.6'}
Common functions for Loader are {'vgetargs1_impl|python3.10', '__GI___libc_malloc|libc.so.6', 'jpeg_idct_16x16|libjpeg.so.9', 'ycc_rgb_convert|libjpeg.so.9', 'jpeg_fill_bit_buffer|libjpeg.so.9', 'PyVectorcall_Function|python3.10', 'PyThread_get_thread_ident|python3.10', '__memmove_avx_unaligned_erms|libc.so.6', '__memset_avx2_unaligned_erms|libc.so.6', 'read|python3.10', '_int_free|libc.so.6', 'jpeg_idct_islow|libjpeg.so.9', '_Py_NewReference|python3.10', '__libc_calloc|libc.so.6', 'decompress_onepass|libjpeg.so.9', 'ImagingUnpackRGB|_imaging.cpython-310-x86_64-linux-gnu.so', 'jpeg_read_scanlines|libjpeg.so.9', 'ImagingJpegDecode|_imaging.cpython-310-x86_64-linux-gnu.so', 'PyEval_Rest

In [8]:
op_to_func

{'RandomHorizontalFlip': ['_PyEval_EvalFrameDefault|python3.10',
  'c10::impl::OperatorEntry::lookup|libtorch_cpu.so',
  'ImagingFlipLeftRight|_imaging.cpython-310-x86_64-linux-gnu.so',
  '__libc_calloc|libc.so.6',
  '__GI___pthread_mutex_lock|libpthread.so.0',
  '__GI___libc_malloc|libc.so.6'],
 'Loader': ['vgetargs1_impl|python3.10',
  '__GI___libc_malloc|libc.so.6',
  'jpeg_idct_16x16|libjpeg.so.9',
  'ycc_rgb_convert|libjpeg.so.9',
  'jpeg_fill_bit_buffer|libjpeg.so.9',
  'PyVectorcall_Function|python3.10',
  'PyThread_get_thread_ident|python3.10',
  '__memmove_avx_unaligned_erms|libc.so.6',
  '__memset_avx2_unaligned_erms|libc.so.6',
  'read|python3.10',
  '_int_free|libc.so.6',
  'jpeg_idct_islow|libjpeg.so.9',
  '_Py_NewReference|python3.10',
  '__libc_calloc|libc.so.6',
  'decompress_onepass|libjpeg.so.9',
  'ImagingUnpackRGB|_imaging.cpython-310-x86_64-linux-gnu.so',
  'jpeg_read_scanlines|libjpeg.so.9',
  'ImagingJpegDecode|_imaging.cpython-310-x86_64-linux-gnu.so',
  'PyEval

In [9]:

def find_common_cpp_func(op_to_func):
    # find functions found in other ops
    common_funcs_to_op = {}
    # loop through all programs
    for python_func in op_to_func.keys():
        # loop through all functions in the program
        for func in op_to_func[python_func]:
            # loop through all other programs
            for other_python_func in op_to_func.keys():
                # skip if the program is the same as the current program
                if other_python_func == python_func:
                    continue
                # if the function is found in other programs
                if func in op_to_func[other_python_func]:
                    # add the function to the set of common functions
                    if func not in common_funcs_to_op:
                        common_funcs_to_op[func] = [python_func,other_python_func]
                    elif python_func not in common_funcs_to_op[func] or other_python_func not in common_funcs_to_op[func]:
                        common_funcs_to_op[func].append(other_python_func)
    return common_funcs_to_op

In [10]:
common_funcs_to_op = find_common_cpp_func(op_to_func)
print("C/C++ functions that can be found in multiple python operations:")
for func in common_funcs_to_op.keys():
    print(f"\t{func}\n\t\t->\t{common_funcs_to_op[func]}")

C/C++ functions that can be found in multiple python operations:
	_PyEval_EvalFrameDefault|python3.10
		->	['RandomHorizontalFlip', 'ToTensor']
	__libc_calloc|libc.so.6
		->	['RandomHorizontalFlip', 'Loader']
	__GI___libc_malloc|libc.so.6
		->	['RandomHorizontalFlip', 'Loader', 'ToTensor']
	vgetargs1_impl|python3.10
		->	['Loader', 'ToTensor']
	__memmove_avx_unaligned_erms|libc.so.6
		->	['Loader', 'RandomResizedCrop', 'ToTensor']
	_int_free|libc.so.6
		->	['Loader', 'RandomResizedCrop', 'Collation', 'ToTensor']
	_Py_NewReference|python3.10
		->	['Loader', 'ToTensor']
	c10::function_ref<void (char**, long const*, long, long)>::callback_fn<at::native::AVX2::VectorizedLoop2d<at::native::AVX2::direct_copy_kernel(at::TensorIteratorBase&)::{lambda()#3}::operator()(void) const::{lambda()#7}::operator()(void) const::{lambda(float)#1}, at::native::AVX2::direct_copy_kernel(at::TensorIteratorBase&)::{lambda()#3}::operator()(void) const::{lambda()#7}::operator()(void) const::{lambda(at::vec::AVX2

In [11]:
json_string = {'op_to_func':op_to_func,'common_funcs_to_op':common_funcs_to_op}
with open('mapping_funcs.json', 'w') as outfile:
    json.dump(json_string, outfile,indent=4)